### Web Crawling
- 웹페이지의 종류
    - 정적 페이지: 페이지의 데이터가 변경될 때 URL이 변경O
    - 동적 페이지: 페이지의 데이터가 변경될 때 URL이 변경X
- requests package
    - 브라우저의 URL을 입력하면 서버에서 데이터를 다운받아 화면에 출력: URL을 입력하면 data를 받아옴.
    - requests package: URL을 입력하면 data를 받아옴.

### Naver Stock Data
- Kospi 지수
- Kosdaq 지수
- USD : 원달러 환율

In [4]:
import pandas as pd
import requests

In [5]:
# 1. 웹서비스를 분석: 크롬 개발자 도구를 사용 -> URL 알아내기

네이버 코스피 https://finance.naver.com/sise/sise_index.naver?code=KOSPI  
1페이지 2페이지 눌러도 URL이 바뀌지 않음 = 동적 페이지 -> JSON 포맷의 데이터를 수집해와야 함  

웹페이지는 정보가 너무 많으니 모바일 페이지로 본다  
https://m.stock.naver.com/domestic/index/KOSPI

In [6]:
url = 'https://m.stock.naver.com/api/index/KOSPI/price?pageSize=10&page=3'

In [7]:
# 2. Request(데이터 요청) > response(데이터 받아오기) => JSON 포맷의 str 타입 데이터를 받아옴.

In [8]:
response = requests.get(url)
print(response) # status code 200 => 잘 가져왔다!

<Response [200]>


In [9]:
print(response.text[:200],'\n')
print(type(response.text)) # str 타입 

[{"localTradedAt":"2022-07-06","closePrice":"2,292.01","compareToPreviousClosePrice":"-49.77","compareToPreviousPrice":{"code":"5","text":"하락","name":"FALLING"},"fluctuationsRatio":"-2.13","openPrice" 

<class 'str'>


In [10]:
# 3. JSON(str) > list, dict > DataFrame으로 변환 

In [11]:
data = response.json()
print(type(data),'\n')
print(data[0])

<class 'list'> 

{'localTradedAt': '2022-07-06', 'closePrice': '2,292.01', 'compareToPreviousClosePrice': '-49.77', 'compareToPreviousPrice': {'code': '5', 'text': '하락', 'name': 'FALLING'}, 'fluctuationsRatio': '-2.13', 'openPrice': '2,330.11', 'highPrice': '2,332.14', 'lowPrice': '2,290.33'}


In [12]:
df = pd.DataFrame(data)[['localTradedAt', 'closePrice']] # 날짜랑 종가 데이터만 가져옴 
df.head()

,localTradedAt,closePrice
0,2022-07-06,"2,292.01"
1,2022-07-05,"2,341.78"
2,2022-07-04,"2,300.34"
3,2022-07-01,"2,305.42"
4,2022-06-30,"2,332.64"


In [13]:
# 4. 함수 만들기
# params = pagesize, page

In [14]:
def stock_price(pagesize, page): # pagesize는 60까지 사용 
    url = f'https://m.stock.naver.com/api/index/KOSPI/price?pageSize={pagesize}&page={page}'
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data)[['localTradedAt', 'closePrice']]
    return df

In [15]:
stock_price(5,2)

,localTradedAt,closePrice
0,2022-07-27,"2,415.53"
1,2022-07-26,"2,412.96"
2,2022-07-25,"2,403.69"
3,2022-07-22,"2,393.14"
4,2022-07-21,"2,409.16"


In [16]:
# KOSDAQ 데이터 수집 코드 작성

In [17]:
# 1. 웹서비스 분석: URL

In [18]:
url = 'https://m.stock.naver.com/api/index/KOSDAQ/price?pageSize=10&page=2'

In [19]:
# 2. Request > response: JSON(str)

In [20]:
response = requests.get(url)
print(response)

<Response [200]>


In [21]:
# 3. JSON(str) > list, dict > DataFrame

In [22]:
data = response.json()
df = pd.DataFrame(data)[['localTradedAt', 'closePrice']]
df.head()

,localTradedAt,closePrice
0,2022-07-20,790.72
1,2022-07-19,782.33
2,2022-07-18,776.72
3,2022-07-15,762.39
4,2022-07-14,766.08


In [23]:
# 4. 함수 만들기
# params = pagesize, page, code = KOSPI

In [24]:
def stock_price(pagesize, page, code='KOSPI'): # pagesize는 60까지 사용 # 기본은 코스피 데이터 가져오기
    # docstring : 함수를 사용하는 방법을 문자열로 작성
    """This function is crawling stock price from naver webpage.
    
    Params
    ------
    pagesize : int : one page size
    page : int : page number
    code : str : KOSPI or KOSDAQ
    
    Return
    ------
    type : DataFrame : display date, price columns
    """
    url = f'https://m.stock.naver.com/api/index/{code}/price?pageSize={pagesize}&page={page}'
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data)[['localTradedAt', 'closePrice']]
    return df

In [25]:
kospi = stock_price(60, 1)
kosdaq = stock_price(60, 1, 'KOSDAQ')

In [26]:
# docstring : 함수를 사용하는 방법을 문자열로 작성
# help()를 쓰거나, 함수 이름(함수 이름만 쓰고)에서 shift + tab
help(stock_price)

Help on function stock_price in module __main__:

stock_price(pagesize, page, code='KOSPI')
    This function is crawling stock price from naver webpage.
    
    Params
    ------
    pagesize : int : one page size
    page : int : page number
    code : str : KOSPI or KOSDAQ
    
    Return
    ------
    type : DataFrame : display date, price columns



* 원달러 환율 데이터 수집 코드 작성

In [27]:
def ExchangeRate(pagesize, page, country = 'USD'):
    url = f'https://api.stock.naver.com/marketindex/exchange/FX_{country}KRW/prices?page={page}&pageSize={pagesize}'
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data)[['localTradedAt', 'closePrice']]
    return df

In [28]:
usd = ExchangeRate(60,1) # 최근 60일치 원달러 환율 데이터
eur = ExchangeRate(60, 1, 'EUR')
jpy = ExchangeRate(60, 1, 'JPY')
cny = ExchangeRate(60, 1, 'CNY')

In [29]:
display(kospi.tail(3))
display(kosdaq.tail(3))
display(usd.tail(3))
display(eur.tail(3))
display(jpy.tail(3))
display(cny.tail(3))

,localTradedAt,closePrice
57,2022-05-12,"2,550.08"
58,2022-05-11,"2,592.27"
59,2022-05-10,"2,596.56"


,localTradedAt,closePrice
57,2022-05-12,833.66
58,2022-05-11,866.34
59,2022-05-10,856.14


,localTradedAt,closePrice
57,2022-05-12,"1,290.50"
58,2022-05-11,"1,274.50"
59,2022-05-10,"1,276.00"


,localTradedAt,closePrice
57,2022-05-12,"1,347.02"
58,2022-05-11,"1,343.77"
59,2022-05-10,"1,346.56"


,localTradedAt,closePrice
57,2022-05-12,"1,002.68"
58,2022-05-11,981.55
59,2022-05-10,980.75


,localTradedAt,closePrice
57,2022-05-12,189.48
58,2022-05-11,189.10
59,2022-05-10,188.97


In [30]:
# 데이터 분석
# 상관 관계 분석: 두 데이터 집합 사이에 어떤 관계가 있는 확인하는 분석 방법
# 원달러 환율이 높으면 코스피, 코스닥 지수가 낮다 = 음의 상관 관계를 갖는다

In [31]:
# 피어슨 상관계수: df.corr()
# 1과 가까울수록 강한 양의 상관 관계를 갖는다.
# -1과 가까울수록 강한 음의 상관 관계를 갖는다.
# 0과 가까울수록 관계가 없다.

In [54]:
# 데이터 전처리
df = kospi.copy()
df['kosdaq'] = kosdaq['closePrice']
df['usd'] = usd['closePrice']
df = df.rename(columns={'closePrice':'kospi'})
df.tail()

,localTradedAt,kospi,kosdaq,usd
55,2022-05-16,"2,596.58",856.25,"1,285.00"
56,2022-05-13,"2,604.24",853.08,"1,284.00"
57,2022-05-12,"2,550.08",833.66,"1,290.50"
58,2022-05-11,"2,592.27",866.34,"1,274.50"
59,2022-05-10,"2,596.56",856.14,"1,276.00"


In [33]:
df[['kospi','kosdaq','usd']].corr() # 아무 것도 안나옴

""


In [34]:
df.dtypes # 왜냐면 type이 문자열이라서... int나 float으로 바꿔줘야함

localTradedAt    object
kospi            object
kosdaq           object
usd              object
dtype: object

In [55]:
# 컬럼 데이터 타입 변경: str > float
# df[column].apply(): 모든 데이터를 함수에 대입한 결과를 출력 
df['kospi'] = df['kospi'].apply(lambda data: float(data.replace(",","")))
df['kosdaq'] = df['kosdaq'].apply(lambda data: float(data.replace(",","")))
df['usd'] = df['usd'].apply(lambda data: float(data.replace(",","")))

In [56]:
df.dtypes

localTradedAt     object
kospi            float64
kosdaq           float64
usd              float64
dtype: object

In [37]:
df[['kospi','kosdaq','usd']].corr()

,kospi,kosdaq,usd
kospi,1.000000,0.984073,-0.878312
kosdaq,0.984073,1.000000,-0.821434
usd,-0.878312,-0.821434,1.000000


In [38]:
# kospi - kosdq : 0.984 => 1과 가까우면 강한 양의 상관 관계 
# kospi - usd : -0.878 => -1과 가까우면 강한 음의 상관 관계 
# ~> 원달러 환율이 높으므로 코스피가 낮다. 그러므로 코스피를 사라..!

In [39]:
# copy(), apply, lambda

In [40]:
data1 = [1, 2, 3]
data2 = data1
print(data1, data2) # [1, 2, 3] [1, 2, 3]

data1[1] = 4
print(data1, data2) # [1, 4, 3] [1, 2, 3]? NO!
# => data1만 바꿨는데 data1과 data2가 다 바뀌었다

[1, 2, 3] [1, 2, 3]
[1, 4, 3] [1, 4, 3]


In [42]:
data1 = [1, 2, 3]
data2 = data1         # 얕은 복사(call by reference): 주소값 복사
data3 = data1.copy()  # 깊은 복사(call by value): 값 복사
print(data1, data2, data3)

data1[1] = 4
print(data1, data2, data3)
# => .copy()를 해서 값을 복사해다 쓰면 원본을 바꿔도 복사한 값이 바뀌지 않는다

[1, 2, 3] [1, 2, 3] [1, 2, 3]
[1, 4, 3] [1, 4, 3] [1, 2, 3]


In [ ]:
# apply(func): 모든 데이터를 func을 적용시킨 결과를 출력

In [43]:
df = pd.DataFrame([{'age':23}, {'age':36}, {'age':27}])
df

,age
0,23
1,36
2,27


In [44]:
# 연령대 컬럼을 추가
def change_ages(age):
    return age // 10 * 10

In [45]:
df['age']

0    23
1    36
2    27
Name: age, dtype: int64

In [49]:
df['ages'] = df['age'].apply(change_ages)
df

,age,ages
0,23,20
1,36,30
2,27,20


In [ ]:
# lambda: 일회성 함수
# 사용 이유: 간단한 함수를 메모리를 절약해서 사용할 수 있다
# 간단한 함수 = 파라미터를 받아서 바로 리턴하는 함수

In [50]:
def plus(n1, n2):
    return n1 + n2

def minus(n1, n2):
    return n1 - n2

def calc(func, n1, n2):
    return func(n1, n2)

# 함수 3개 사용 = 메모리 3칸 사용

In [51]:
calc(plus, 1, 2), calc(minus, 1, 2)

(3, -1)

In [52]:
# lambda 변수 : return할 값
plus_lambda = lambda n1, n2: n1 + n2
plus(2,3), plus_lambda(2,3)

(5, 5)

In [53]:
def calc(func, n1, n2): return func(n1, n2)

calc(lambda n1, n2: n1 + n2, 1, 2), calc(lambda n1, n2: n1 - n2, 1, 2)
# 함수 1개 사용 = 메모리 1칸 사용

(3, -1)

In [ ]:
# summary
# 웹페이지의 종류
# - 정적 페이지: 데이터가 변경될 때 URL 변경 O: HTML 포맷 데이터로 수집
# - 동적 페이지: 데이터가 변경될 때 URL 변경 X: JSON 포맷 데이터로 수집

# 웹크롤링 절차
# 1. 웹서비스 분석: 크롬 개발자 도구 사용: URL을 찾아냄
# 2. requests(url) > response(json) => JSON(str)
# 3. JSON(str) > list or dict > DataFrame으로 변환